# 06-05-02

- Continous Features
- Transition Matrices
- Euclidean Distance
- Min Max
- Remove Constant Features

In [24]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures, scalerMinMax

In [25]:
cover = loadCoverGroups()
listCovers = cover[12]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [26]:
continousFeatures = pd.read_csv('./data/unified/05-continousFeatures.csv')
continousFeatures = continousFeatures.sort_values('id')
continousFeatures.id = continousFeatures.id.astype('int64')
continousFeatures = continousFeatures.set_index('id')

In [27]:
continousFeatures = continousFeatures.iloc[:,1:]
continousFeatures

,danceability,energy,speechiness,acousticness,liveness,valence,tempo
id,,,,,,,
23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337
105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344
118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622
120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097
292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932
...,...,...,...,...,...,...,...
6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030
6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984
6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089


In [28]:
transitionMatrices = pd.read_csv('./data/unified/transitionMatrices.csv')
transitionMatrices = transitionMatrices.iloc[:, 1:]
transitionMatrices = transitionMatrices.sort_values('id')
transitionMatrices.id = transitionMatrices.id.astype('int64')
transitionMatrices = transitionMatrices.set_index('id')

In [29]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0


In [30]:
df = transitionMatrices.merge(continousFeatures, left_index=True, right_index=True)
df

,0,1,2,3,4,5,6,7,8,9,...,1293,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.00000,0.0,0.0,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089


In [31]:
df['id'] = df.index

In [32]:
df[df.id == originalSongId]

,0,1,2,3,4,5,6,7,8,9,...,1294,1295,danceability,energy,speechiness,acousticness,liveness,valence,tempo,id
id,,,,,,,,,,,,,,,,,,,,,
6001918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.683,0.691,0.0432,0.243,0.14,0.732,179.91,6001918


In [33]:
df.reset_index(drop=True, inplace=True)

In [34]:
ids = df.id
df = df.iloc[:, 0:-1]

In [35]:
df = removeConstantFeatures(df)
df = scalerMinMax(df)

## Applying eucledian distance

In [36]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [37]:
originalSongIndex = ids[ids == originalSongId].index[0]
originalSongIndex

originalSongVector = df.iloc[originalSongIndex, :]

***Dropping the cover song***

In [38]:
coverIndex = ids[ids == coverSongId].index[0]
coverSongVector = df.iloc[coverIndex, :]

In [39]:
ranking = []
for index in range(0, len(df)):
    ranking.append([ids[index], euclidean_distance(df.iloc[index, :], originalSongVector)])

In [40]:
ranksDF = pd.DataFrame(ranking)

In [41]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [42]:
ranksDF = ranksDF.sort_values(by="rank")

In [43]:
ranksDF.reset_index(drop=True, inplace=True)

In [44]:
ranksDF.head(10)

,id,rank
0,6001918,0.000000
1,6001599,0.746305
2,6000373,0.765191
3,6001574,0.775201
4,6002002,0.870481
5,6001782,0.873839
6,6001512,0.878100
7,6001270,0.911766
8,6001758,0.916311
9,970956,0.922454


In [45]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
3572,6001997,1.642246


In [46]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,6001918,0.000000
3572,6001997,1.642246
